In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('../단기모멘텀.csv', dtype={'ticker':str})
df2 = pd.read_csv('../24.12_수급,거래 데이터.csv', dtype={'ticker':str})

In [3]:
# ------------------------------------------------------------------------------
# 0. 데이터 준비
# ------------------------------------------------------------------------------
print("🚀 1. 데이터 병합 및 초기화...")
df1['date'] = pd.to_datetime(df1['date'])
df2['date'] = pd.to_datetime(df2['date'])
df1['ticker'] = df1['ticker'].astype(str)
df2['ticker'] = df2['ticker'].astype(str)

df = pd.merge(df1, df2, on=['date', 'ticker'], how='left')
inv_cols = [c for c in df.columns if '순매수' in c]
df[inv_cols] = df[inv_cols].fillna(0)
df = df.sort_values(['ticker', 'date']).reset_index(drop=True)

🚀 1. 데이터 병합 및 초기화...


In [4]:

# ------------------------------------------------------------------------------
# 1. Feature Engineering (120일 지표 추가 - 성능 강화)
# ------------------------------------------------------------------------------
print("🚀 2. 핵심 지표 생성 (60일, 120일 이격도 포함)...")

# (1) 이격도 (Disparity): 5, 20, 60, 120일
# * 60일(수급선), 120일(경기선) 추가 -> 장기 추세 대비 현재 위치 파악
# * 120일 이격도가 낮으면 '역사적 저점'일 가능성이 높음
for w in [5, 20, 60, 120]:
    ma = df.groupby('ticker')['종가'].rolling(w).mean().reset_index(0, drop=True)
    df[f'disparity_{w}'] = df['종가'] / ma

# (2) RSI (14일)
def calc_rsi(x, w=14):
    delta = x.diff()
    gain = (delta.where(delta > 0, 0)).rolling(w).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(w).mean()
    rs = gain / (loss + 1e-9)
    return 100 - (100 / (1 + rs))
df['rsi_14'] = df.groupby('ticker')['종가'].transform(lambda x: calc_rsi(x))

# (3) 볼린저 밴드 %B
def calc_bb_pb(x, w=20):
    m = x.rolling(w).mean()
    s = x.rolling(w).std()
    lower = m - 2*s
    upper = m + 2*s
    return (x - lower) / (upper - lower + 1e-9)
df['bollinger_pb'] = df.groupby('ticker')['종가'].transform(lambda x: calc_bb_pb(x))

# (4) 변동성 (5일)
df['volatility_5'] = df.groupby('ticker')['등락률'].rolling(5).std().reset_index(0, drop=True)

# (5) 거래량 비율
v5 = df.groupby('ticker')['거래량'].rolling(5).mean().reset_index(0, drop=True)
v20 = df.groupby('ticker')['거래량'].rolling(20).mean().reset_index(0, drop=True)
df['volume_ratio'] = v5 / (v20 + 1e-9)

# (6) 수급 누적 (5일)
target_amt = '순매수_금액' if '외국인_순매수_금액' in df.columns else '순매수_수량'
for subj in ['외국인', '기관', '개인']:
    col_name = f'{subj}_{target_amt}'
    if col_name in df.columns:
        df[f'{col_name}_sum_5'] = df.groupby('ticker')[col_name].rolling(5).sum().reset_index(0, drop=True)

🚀 2. 핵심 지표 생성 (60일, 120일 이격도 포함)...


In [5]:

# ------------------------------------------------------------------------------
# 2. Market Filter용 지표 & Target
# ------------------------------------------------------------------------------
# 60일 이평선 (하락장 방어용)
df['ma_60'] = df.groupby('ticker')['종가'].transform(lambda x: x.rolling(60).mean())
df['above_ma60'] = np.where(df['종가'] > df['ma_60'], 1, 0)

# Target: 10일 후 수익률 순위
df['fwd_ret_10d'] = df.groupby('ticker')['종가'].shift(-10) / df['종가'] - 1
df['target_rank'] = df.groupby('date')['fwd_ret_10d'].rank(pct=True)

In [6]:
df

,date,시가,고가,저가,종가,거래량,등락률,ticker,ret_1d,ret_5d,...,bollinger_pb,volatility_5,volume_ratio,외국인_순매수_금액_sum_5,기관_순매수_금액_sum_5,개인_순매수_금액_sum_5,ma_60,above_ma60,fwd_ret_10d,target_rank
0,2015-01-02,23100,23100,22350,22600,184419,-2.795699,000080,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,-0.004425,0.559006
1,2015-01-05,22600,23150,22400,23050,170602,1.991150,000080,0.019912,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,-0.028200,0.322981
2,2015-01-06,22750,23100,22550,22650,142141,-1.735358,000080,-0.017354,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.030905,0.596273
3,2015-01-07,22650,23050,22350,22700,159007,0.220751,000080,0.002208,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.011013,0.472050
4,2015-01-08,22600,23100,22600,22850,126066,0.660793,000080,0.006608,NaN,...,NaN,1.918816,NaN,2.185804e+08,-1.364644e+08,1.125672e+08,NaN,0,0.000000,0.406832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443040,2024-12-23,29100,30100,29050,29550,888310,2.961672,489790,0.029617,-0.166432,...,0.003569,4.981788,0.917643,-7.444723e+09,-1.651676e+10,2.595550e+10,NaN,0,NaN,NaN
443041,2024-12-24,29950,31350,29100,30400,984329,2.876481,489790,0.028765,-0.136364,...,0.138029,5.686780,0.943990,-4.607088e+09,-1.895513e+10,2.202578e+10,38303.333333,0,NaN,NaN
443042,2024-12-26,30900,31450,30300,30600,557107,0.657895,489790,0.006579,-0.102639,...,0.195202,5.873980,1.019024,-9.609240e+08,-1.408950e+10,1.417099e+10,38223.333333,0,NaN,NaN
443043,2024-12-27,30150,32100,29700,31150,991359,1.797386,489790,0.017974,-0.020440,...,0.278939,5.369299,0.933772,5.152243e+09,-1.084868e+09,-4.089085e+09,38187.500000,0,NaN,NaN


In [7]:
df.to_csv('../단기모델1.csv')

In [8]:
# ------------------------------------------------------------------------------
# 3. 데이터 정리
# ------------------------------------------------------------------------------
# [수정] 60, 120일 이격도 추가
features = [
    'disparity_5', 'disparity_20', 'disparity_60', 'disparity_120', # 장기 이격도 추가
    'rsi_14', 'bollinger_pb', 
    'volatility_5', 'volume_ratio', 'ret_1d', 'ret_20d', 'ret_5d'
] + [c for c in df.columns if '_sum_5' in c]

# 결측치 제거 
# 주의: 120일치(약 6개월) 데이터가 NaN이라 삭제됩니다. (2015년 1~6월 데이터 삭제)
model_df = df.dropna(subset=features + ['target_rank', 'above_ma60']).copy()

print(f"✅ 컬럼 생성 완료! (120일 이격도 포함)")
print(f"👉 사용 Feature ({len(features)}개): {features}")
print(f"👉 데이터 시작일: {model_df['date'].min()} (초기 6개월 삭제됨)")

✅ 컬럼 생성 완료! (120일 이격도 포함)
👉 사용 Feature (14개): ['disparity_5', 'disparity_20', 'disparity_60', 'disparity_120', 'rsi_14', 'bollinger_pb', 'volatility_5', 'volume_ratio', 'ret_1d', 'ret_20d', 'ret_5d', '외국인_순매수_금액_sum_5', '기관_순매수_금액_sum_5', '개인_순매수_금액_sum_5']
👉 데이터 시작일: 2015-06-26 00:00:00 (초기 6개월 삭제됨)
